# Import libraries

In [1]:
import pandas as pd
import numpy as np
import os
import fasttext
import pandas as pd
import gensim
from gensim.test.utils import get_tmpfile
from gensim.models.word2vec import Word2Vec
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
import time
from keras import backend as K

Using TensorFlow backend.


# Parameters

In [2]:
lstm_neurons = 80
SAVE_PROGRESSIVE = False

#THE PAPER WAS TRAINED USING VECSIZE 100 AND WINDOWSIZE 2
vecSize = 100 
windowSize = 2
timesteps = 100
pretrainedModelDimms = 300
keras_iterations = 30
batch_size = 128
flag_verbose = 1

# DEFINE PRETRAINED MODEL

#from gensim.models import KeyedVectors
#fname="GoogleNews-vectors-negative300.bin"
fname_pretrained="crawl-300d-2M.vec"

if not os.path.exists(fname_pretrained):
    !wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
    !unzip crawl-300d-2M.vec.zip

# Load Dataset

In [3]:
df_overall = pd.read_csv("dataset.csv", index_col=0)
df_overall.head()

tags = list(df_overall.tags.unique())
ref = {}
index=1
for tag in tags:
    if str(tag) == 'nan':
        continue
    ref[tag]=index
    index+=1
print(ref)

{'c#': 1, 'sql': 9, 'javascript': 7, 'php': 10, 'r': 4, 'perl': 6, 'python': 3, 'java': 2, 'vb.net': 5, 'c++': 8}


# Plow the text from the dataset into a text file, and the train a custom model on that text

In [4]:
sentences = np.array(df_overall["body"]).tolist()

for i in range(len(sentences)):
    sentences[i] = str(sentences[i])
training_string = '\n'.join(sentences)

with open("training_file.txt", "w") as text_file:
    text_file.write(training_string)

print("Training Custom Model..")

# customw2v = fasttext.skipgram('training_file.txt', 'model', ws=windowSize, dim=vecSize, min_count=3)
model_custom = fasttext.train_unsupervised('training_file.txt', model='skipgram', dim=vecSize, ws=windowSize, epoch=5)
# customw2v = Word2Vec(sentences, size=vecSize, window=windowSize, min_count=3, workers=4)

fname = get_tmpfile("model_custom.model")
model_custom.save_model(fname)
model_custom = fasttext.load_model(fname)
print("Done..")

Training Custom Model..
Done..


# Define method to get custom vectors from custom language model

In [5]:
def sentVectorsCustom(keywordListIn):
    global model_custom
    sentenceVectorComponents = []
    
    pattern_vector = False
    n_words = 0
    for keyword in keywordListIn:
        #try splitting into two words
        if not keyword in model_custom:
            for splitIndex in range(len(keyword)):
                firstWord=keyword[0:splitIndex]
                secondWord=keyword[splitIndex:]
                if(firstWord in model_custom):
                    vector = model_custom[firstWord]
                    sentenceVectorComponents.append(vector)
                    n_words+=1 #NOTE: this case messes with the ratio a bit. One word can be counted as 2 after it is unfused
                    break
                if(secondWord in model_custom):
                    vector = model_custom[secondWord]
                    sentenceVectorComponents.append(vector)
                    n_words+=1 #NOTE: this case messes with the ratio a bit. One word can be counted as 2 after it is unfused
                    break
                
        if(keyword in model_custom):
            vector = model_custom[keyword]
            sentenceVectorComponents.append(vector)
            n_words += 1
    #print(n_words,pattern_vector)
    return sentenceVectorComponents #pattern_vector    #,n_words,

def embedCustom(row):
    global popo
    popo += 1
    if(popo % 1000 == 0):
        print("row " + str(popo))
    return [sentVectorsCustom(str(row["body"]).split())]

print("Starting Custom Vectors Generation")
popo = 0
df_overall['vector_custom'] = df_overall.apply(embedCustom, axis=1)
del model_custom
print("Custom Vectors Generated.")

Starting Custom Vectors Generation
row 1000
row 2000
row 3000
row 4000
row 5000
row 6000
row 7000
row 8000
row 9000
row 10000
Custom Vectors Generated.


# Load pre-trained model and get vectors from it

In [6]:
print("Loading Pretrained Model..")
model_pretrained = gensim.models.KeyedVectors.load_word2vec_format(fname_pretrained, binary=False) #false for fasttext & true for googleNews
# model_pretrained = fasttext.load_model(fname_pretrained)
print("Pretrained Model Loaded.")

def avgVectors(keywordListIn):
    global model_pretrained
    sentenceVectorComponents = []
    
    n_words = 0
    for keyword in keywordListIn:
        
        #try splitting into two words
        if not keyword in model_pretrained:
            for splitIndex in range(len(keyword)):
                firstWord=keyword[0:splitIndex]
                secondWord=keyword[splitIndex:]
                #print(firstWord,secondWord)
                if(firstWord in model_pretrained):
                    vector = model_pretrained[firstWord]
                    sentenceVectorComponents.append(vector)
                    n_words+=1 #NOTE: this case messes with the ratio a bit. One word can be counted as 2 after it is unfused
                    break
                if(secondWord in model_pretrained):
                    vector = model_pretrained[secondWord]
                    sentenceVectorComponents.append(vector)
                    n_words+=1 #NOTE: this case messes with the ratio a bit. One word can be counted as 2 after it is unfused
                    break
                
        if(keyword in model_pretrained):
            vector = model_pretrained[keyword]
            sentenceVectorComponents.append(vector)
            n_words += 1

    #print(n_words,pattern_vector)
    return sentenceVectorComponents #pattern_vector    #,n_words,

def embed(row):
    global popo
    popo += 1
    if(popo % 1000 == 0):
        print("row " + str(popo))
    return [avgVectors(str(row["body"]).split())]

print("Starting Pretrained Vectors Generation")
popo = 0
df_overall['vector'] = df_overall.apply(embed, axis=1)
del model_pretrained
print("Pretrained Vectors Generated")

Loading Pretrained Model..
Pretrained Model Loaded.
Starting Pretrained Vectors Generation
row 1000
row 2000
row 3000
row 4000
row 5000
row 6000
row 7000
row 8000
row 9000
row 10000
Pretrained Vectors Generated


In [7]:
df_overall.to_pickle('df_overall_fastText_vectors.p')
display(df_overall.info(memory_usage='deep'))
display(len(df_overall))

<class 'pandas.core.frame.DataFrame'>
Index: 10001 entries, 0 to 999
Data columns (total 5 columns):
[LocalizedFileNames]    1 non-null float64
body                    10000 non-null object
tags                    10000 non-null object
vector_custom           10001 non-null object
vector                  10001 non-null object
dtypes: float64(1), object(4)
memory usage: 14.2 MB


None

10001

In [8]:
df_overall = pd.read_pickle('df_overall_fastText_vectors.p')
df_overall.head()

,[LocalizedFileNames],body,tags,vector_custom,vector
0,NaN,<p>How i can convert word file (.docx &amp; do...,c#,"[[[0.46398896, 0.16047312, -0.5810341, 0.00351...","[[[0.3435, 0.6339, -0.0938, -0.1206, -0.0436, ..."
1,NaN,<p>I essentially have the following:</p>\n\n<p...,c#,"[[[-0.27910337, 0.9925494, -0.39375988, 0.4042...","[[[-0.7691, -0.2869, 0.0241, 0.2612, 0.17, 0.2..."
2,NaN,<p>I have a variable which I am populating wit...,c#,"[[[-0.27910337, 0.9925494, -0.39375988, 0.4042...","[[[-0.7691, -0.2869, 0.0241, 0.2612, 0.17, 0.2..."
3,NaN,<pre><code>public new int AdjustedBaseValue\n<...,c#,"[[[-0.39814365, 0.20550673, 0.40518367, -0.452...","[[[0.3435, 0.6339, -0.0938, -0.1206, -0.0436, ..."
4,NaN,"<p>Sorry for this question, but I've been Goog...",c#,"[[[0.2184001, 0.52190584, -0.6915303, 0.285176...","[[[0.3435, 0.6339, -0.0938, -0.1206, -0.0436, ..."


# PRE-TRAINED WORD EMBEDDING X AND Y CREATION

In [9]:
Y = pd.get_dummies(df_overall['tags']).fillna(0)
print("Y.shape = " + str(Y.shape))
Y = np.array(Y)

X = df_overall[["vector"]].fillna(0)
rows=X.shape[0]
print("Xshape = " + str(np.array(np.array(X)[0][0]).shape))
X=np.array(X)

def column(matrix, i):
    return (row[i] for row in matrix)

#X[i][2][0]
from keras.preprocessing.sequence import pad_sequences
newarr=np.empty([rows,timesteps,pretrainedModelDimms])
for i in range(rows):
    if(i%1000 == 0):
        print("row " + str(i))
    if len(X[i][0][0])==0:
        xs=np.zeros([1,timesteps,pretrainedModelDimms])
    else:
        xd=np.stack(X[i][0][0], axis=0)
        xd=xd.swapaxes(0,1)#np.swapaxes(xd,0,1)
        xs=pad_sequences(xd, maxlen=timesteps, dtype='float32')
        xs=np.swapaxes(xs,0,1)
    print("xs.shape = " + str(xs.shape))
    print("newarr[i].shape = " + str(newarr[i].shape))
    newarr[i]=xs
X=newarr
print("X.shape = " + str(X.shape))

Y.shape = (10001, 10)
Xshape = (1, 26, 300)
row 0
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].

xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i]

xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i]

xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i]

xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i]

...

xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i]

xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i]

xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i]

xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i]

xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i].shape = (100, 300)
xs.shape = (100, 300)
newarr[i]

In [18]:
print("Xshape = " + str(np.array(np.array(X)[0][0]).shape[0]))

Xshape = 400


In [11]:
np.count_nonzero(X)

242869764

In [12]:
#CUSTOM WORD EMBEDDING X AND Y CREATION
X2 = df_overall[["vector_custom"]].fillna(0)
rows=X2.shape[0]
X2=np.array(X2)

from keras.preprocessing.sequence import pad_sequences
newarr=np.empty([rows,timesteps,vecSize])
for i in range(rows):
    if len(X2[i][0][0])==0:
        xs=np.zeros([1,timesteps,vecSize])
    else:
        xd=np.stack(X2[i][0][0], axis=0)
        xd=np.swapaxes(xd,0,1)
        xs=pad_sequences(xd, maxlen=vecSize, dtype='float32')
        xs=np.swapaxes(xs,0,1)
    newarr[i]=xs

X2=newarr
X=np.swapaxes(X,1,2)
X2=np.swapaxes(X2,1,2)
X = np.hstack((X,X2))
X=np.swapaxes(X,1,2)

print(X.shape,Y.shape)

(10001, 100, 400) (10001, 10)


In [13]:
np.count_nonzero(X)

323849264

# TEST/TRAIN SPLIT ON concatenated PRE-TRAINED + CUSTOM vectors

In [14]:
x_train, x_test, y_train, y_test=train_test_split(X, Y, test_size=0.05, random_state=42)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

data_dim = pretrainedModelDimms + vecSize
num_classes = len(ref.keys())

(9500, 100, 400) (501, 100, 400) (9500, 10) (501, 10)


In [15]:
K.clear_session()
print(str(lstm_neurons) + " LSTM neurons ")
model = Sequential()
model.add(LSTM(lstm_neurons, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
model.add(LSTM(lstm_neurons, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(lstm_neurons))  # return a single vector of dimension 32
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
import time
for i in range(keras_iterations):
    print('Iteration: '+str(i))
    model.fit(x_train, y_train, epochs=1,  batch_size=batch_size, verbose=flag_verbose)#, class_weight="auto"
    score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=flag_verbose)
    print("SCORE: {0:.3f}".format(score[1]))
    if i == 20 and SAVE_PROGRESSIVE:
        model.save('model_' + str(lstm_neurons) + '_20_iter.h5')

fname_sidecar='model_' + str(lstm_neurons) + '.h5'
model.save(fname_sidecar)
print("Ok.")

80 LSTM neurons 
Iteration: 0
Epoch 1/1
501/501 [==============================] - 2s 3ms/step
SCORE: 0.465
Iteration: 1
Epoch 1/1
501/501 [==============================] - 1s 2ms/step
SCORE: 0.569
Iteration: 2
Epoch 1/1
501/501 [==============================] - 1s 2ms/step
SCORE: 0.631
Iteration: 3
Epoch 1/1
501/501 [==============================] - 1s 2ms/step
SCORE: 0.649
Iteration: 4
Epoch 1/1
501/501 [==============================] - 1s 2ms/step
SCORE: 0.645
Iteration: 5
Epoch 1/1
501/501 [==============================] - 1s 2ms/step
SCORE: 0.655
Iteration: 6
Epoch 1/1
501/501 [==============================] - 1s 2ms/step
SCORE: 0.705
Iteration: 7
Epoch 1/1
501/501 [==============================] - 1s 2ms/step
SCORE: 0.699
Iteration: 8
Epoch 1/1
501/501 [==============================] - 1s 2ms/step
SCORE: 0.709
Iteration: 9
Epoch 1/1
501/501 [==============================] - 1s 2ms/step
SCORE: 0.721
Iteration: 10
Epoch 1/1
501/501 [==============================] - 1s 2ms

In [16]:
K.clear_session()
print(fname_sidecar)
model = keras.models.load_model(fname_sidecar)
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=flag_verbose)
print(np.around(score[1], decimals=3))

model_80.h5
501/501 [==============================] - 1s 3ms/step
0.78


In [17]:
fname_sidecar='model_' + str(lstm_neurons) + '.h5'
model.save(fname_sidecar)
print("Ok.")

Ok.
